In [11]:
# Imports

import torch
from torchvision import models, transforms
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display, clear_output
from jetbot import Camera, bgr8_to_jpeg
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import time
import math
from jetbot import Robot
import utils
import threading
import map_utils
import sys

Succeeded to open the port
Succeeded to change the baudrate


In [10]:
import cv2, time, traitlets, threading
import ipywidgets as widgets
from IPython.display import display

class CameraProvider(traitlets.HasTraits):
    value = traitlets.Bytes()  # przechowuje bajty JPEG

    def __init__(self, rtsp_url):
        super().__init__()
        self.cap = cv2.VideoCapture(rtsp_url)
        self.running = False

    def start(self):
        self.running = True
        def update_loop():
            while self.running:
                ret, frame = self.cap.read()
                if ret:
                    # Kodowanie klatki do JPEG
                    _, buf = cv2.imencode('.jpeg', frame)
                    self.value = buf.tobytes()
                time.sleep(0.04)  # ograniczanie do ~25 FPS
        threading.Thread(target=update_loop, daemon=True).start()

    def stop(self):
        self.running = False
        if self.cap.isOpened():
            self.cap.release()

# Inicjalizacja klasy "Provider" z adresem RTSP
camera_provider = CameraProvider("rtsp://192.168.16.148:8080/h264.sdp")

# Widget obrazu - będzie odświeżany w czasie rzeczywistym
camera_widget = widgets.Image(format='jpeg')  # można dodać width/height, np. width='400px'

# Połączenie (dlink) atrybutu "value" z widgetem
traitlets.dlink((camera_provider, 'value'), (camera_widget, 'value'))

# Uruchomienie aktualizacji i wyświetlenie widgetu
camera_provider.start()
display(camera_widget)


Image(value=b'', format='jpeg')

In [12]:
# Widget kamera 
camera = Camera.instance(width=300, height=300)
image = widgets.Image(format='jpeg', width=300, height=300)  # this width and height doesn't necessarily have to match the camera
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [13]:
# Model rozpoznawania podłogi
floor_prototype = np.load("../models/floor_prototype_full.npy")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)
model.fc = torch.nn.Identity()  # Usunięcie ostatniej warstwy
model = model.to(device)
model.eval()  # Tryb ewaluacji

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [14]:
# Robot
robot = Robot()

# Parametry sterowania
FORWARD_SPEED = 0.5
TURN_SPEED = 0.5
DELAY = 0.1     # Czas odświeżania pętli w sekundach
MAP_SIZE = 400  # Rozmiar mapy
CELL_SIZE = 0.01  # 1 pixel = 0.01 m
ROBOT_SIZE = 0.2  # Rozmiar robota w metrach
VISION_RANGE = 0.2  # Jak daleko widzi robot (0.2m = 20 pikseli)
FORWARD_SPEED_MS = FORWARD_SPEED * 0.16  # Prędkość w metrach na sekundę
TURN_ANGLE = 30  # Robot skręca o 30 stopni przy losowym wyborze kierunku

# Inicjalizacja mapy (-1 = nieznane)
map_grid = np.full((MAP_SIZE, MAP_SIZE), -1)

# Pozycja początkowa robota w środku mapy
robot_x, robot_y = MAP_SIZE // 2, MAP_SIZE // 2
robot_direction = 0  # Kąt obrotu w stopniach (0 = góra)

In [16]:
# Widget Mapa
map_provider = map_utils.MapProvider(map_size=MAP_SIZE)
map_widget = widgets.Image(format='jpeg', width=300, height=300)
map_link = traitlets.dlink((map_provider, 'value'), (map_widget, 'value'))

map_provider.set_robot_pos(robot_x, robot_y)
map_provider.set_map(map_grid)

display(map_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [17]:
def move_and_print(distance=sys.maxsize):
    global robot_direction, robot_x, robot_y
    last_move_time = time.time()  # Czas ostatniego ruchu do przodu
    previous_move_was_forward = False  # Czy ostatni ruch był do przodu?
    try:
        #print(f"START X: {robot_x}    Y:{robot_y}")
        has_turned = False  # Flaga: Czy robot ostatnio skręcał?
        turn_direction = None  # Kierunek ostatniego skrętu: 'left' lub 'right'

        # while True
        for step in range(distance):
            print(f"STEP {step:4d}: X: {robot_x:6.2f}, Y: {robot_y:6.2f}, direction: {robot_direction:6.2f}")
            image = Image.fromarray(camera.value)
            forward_distance = utils.is_floor(image, model, device, floor_prototype)

            if previous_move_was_forward:
                current_time = time.time()
                time_elapsed = current_time - last_move_time
                last_move_time = current_time  # Aktualizacja ostatniego ruchu
                step_size = FORWARD_SPEED_MS * time_elapsed / CELL_SIZE
                # Obliczenie nowej pozycji robota
                angle_rad = np.radians(robot_direction)
                robot_y += step_size * np.cos(angle_rad)
                robot_x += step_size * np.sin(angle_rad)

            if forward_distance < 0.175: #and not utils.any_red_20_20(map_grid, robot_direction, robot_x, robot_y):  # Jeśli podłoga wykryta
                #print(f"Podłoga wykryta. Jedź do przodu. Odległość: {forward_distance:.4f}")
                map_provider.update_map_grid(forward_distance, int(robot_x), int(robot_y), robot_direction, map_grid)
                utils.move_forward(robot, FORWARD_SPEED)
                last_move_time = time.time()
                time.sleep(DELAY)  # Jedź chwilę do przodu
                has_turned = False
                turn_direction = None
                previous_move_was_forward = True
            else:
                #print(f"Brak podłogi przed robotem. Odległość: {forward_distance:.4f}")
                previous_move_was_forward = False
                utils.stop(robot)
                if has_turned:
                    pass
                    #print(f"Kontynuowanie skrętu w kierunku: {turn_direction}")
                else:
                    # Rozejrzenie się i wybór najlepszego kierunku
                    #print("Rozejrzenie się...")
                    utils.turn_by_angle(robot, -TURN_ANGLE/2, TURN_SPEED)  # Obrót w lewo o 15°
                    image = Image.fromarray(camera.value)
                    left_distance = utils.is_floor(image, model, device, floor_prototype)
                    utils.turn_by_angle(robot, TURN_ANGLE, TURN_SPEED)  # Obrót w prawo o 30° (15° w prawo od pozycji wyjściowej)
                    image = Image.fromarray(camera.value)
                    right_distance = utils.is_floor(image, model, device, floor_prototype)
                    # Powrót do pozycji początkowej (15° w lewo od pozycji prawej)
                    utils.turn_by_angle(robot, -TURN_ANGLE/2, TURN_SPEED)
                    # Wybór kierunku
                    if left_distance < right_distance:
                        #print(f"Skręt w lewo o {TURN_ANGLE}°. Odległość w lewo: {left_distance:.4f}")
                        turn_direction = 'left'
                        forward_distance = left_distance
                    else:
                        #print(f"Skręt w prawo o {TURN_ANGLE}°. Odległość w prawo: {right_distance:.4f}")
                        turn_direction = 'right'
                        forward_distance = right_distance
                    has_turned = True

                if turn_direction == 'left':
                    map_provider.update_map_grid(forward_distance, int(robot_x), int(robot_y), robot_direction, map_grid)
                    utils.turn_by_angle(robot, -TURN_ANGLE, TURN_SPEED)
                    robot_direction = (robot_direction - TURN_ANGLE) % 360
                elif turn_direction == 'right':
                    map_provider.update_map_grid(forward_distance, int(robot_x), int(robot_y), robot_direction, map_grid)
                    utils.turn_by_angle(robot, TURN_ANGLE, TURN_SPEED)
                    robot_direction = (robot_direction + TURN_ANGLE) % 360
            
            #if i % 10 ==0:
            map_provider.set_robot_pos(robot_x, robot_y)
            map_provider.set_map(map_grid)

    except KeyboardInterrupt:
        print("Zatrzymano robota.")
    finally:
        utils.stop(robot)

    


In [19]:
move_and_print(200)

STEP    0: X: 138.44, Y: 233.56, direction: 150.00
STEP    1: X: 138.44, Y: 233.56, direction: 150.00
STEP    2: X: 139.24, Y: 232.17, direction: 150.00
STEP    3: X: 140.06, Y: 230.74, direction: 150.00
STEP    4: X: 140.88, Y: 229.33, direction: 150.00
STEP    5: X: 141.63, Y: 228.03, direction: 150.00
STEP    6: X: 142.39, Y: 226.71, direction: 150.00
STEP    7: X: 143.11, Y: 225.46, direction: 150.00
STEP    8: X: 143.89, Y: 224.11, direction: 150.00
STEP    9: X: 144.74, Y: 222.64, direction: 150.00
STEP   10: X: 145.48, Y: 221.35, direction: 150.00
STEP   11: X: 146.25, Y: 220.02, direction: 150.00
STEP   12: X: 147.03, Y: 218.67, direction: 150.00
STEP   13: X: 147.79, Y: 217.36, direction: 180.00
STEP   14: X: 147.79, Y: 217.36, direction: 180.00
STEP   15: X: 147.79, Y: 215.80, direction: 180.00
STEP   16: X: 147.79, Y: 214.26, direction: 180.00
STEP   17: X: 147.79, Y: 212.73, direction: 180.00
STEP   18: X: 147.79, Y: 211.14, direction: 180.00
STEP   19: X: 147.79, Y: 209.39

In [ ]:
camera.stop()